# 构建流程 

In [77]:
from bs4 import BeautifulSoup
import urllib
def main():
    baseurl = "https://movie.douban.com/top250?start="
    askURL(baseurl)
    getData(baseurl)
   
#1.爬取网页
#     datalist = getData(baseurl)
#     savepath = ".\\豆瓣电影Top50.xls"

#影片详情
findLink = re.compile(r'<a href="(.*?)">') #创建正则对象 表示规则
#影片图片
findImgSrc = re.compile(r'<img.*src="(.*?)"',re.S) #不是我们查找的内容 不加括号
                                                   #re.S让换行符包含其中
#影片片名
findTitle = re.compile(r'<span class="title">(.*?)</span>')
#影片评分
findScore = re.compile(r'<span class="rating_num" property="v:average">(.*)</span>')
#评价人数
findNum = re.compile(r'<span>(\d*)人评价</span>')
#找到概况
findInq = re.compile(r'<span class="inq">(.*?)</span>')
#找到影片相关内容
findBd = re.compile(r'<p class="">(.*?)</p>',re.S)


#爬取网页
def getData(baseurl):
    datalist = []
    for i in range(0,1):     #调用获取页面信息的函数：10次
        url = baseurl+str(i*25)
        html = askURL(url)
        
        #2.逐一解析数据
        soup = BeautifulSoup(html,"html.parser")
        for temp in soup.find_all("div",class_="item"): #查找符合要求的字符串
            # print(temp) 测试查看item信息
            data = [] #保存库中所有信息
            temp = str(temp)
            
            #获取影片详情链接
            link = re.findall(findLink,temp)[0] #按照findLink规则搜索temp
            data.append(link)
            
            imgSrc = re.findall(findImgSrc,temp)[0]
            data.append(imgSrc)
            
            titles = re.findall(findTitle,temp) #title可能有中文 可能有英文
            if(len(titles) == 2):
                ctitle = titles[0]
                data.append(ctitle)
                otitle = re.sub("/","",titles[1]) #去掉无关符号
                data.append(otitle)
            else:
                data.append(titles[0])
                data.append(" ")     #外国名留空
            
            score = re.findall(findScore,temp)[0]
            data.append(score)
            
            Num = re.findall(findNum,temp)[0]
            data.append(Num)
            
            inq = re.findall(findInq,temp)[0]
            if len(inq) != 0:
                inq = re.sub("。","",inq)
                data.append(inq)
            else:
                data.append(" ")
                
            bd = re.findall(findBd,temp)[0]
            bd = re.sub('<br(\s+)?/>(\s+)?'," ",bd) #去掉<br/>及其相关字符
            bd = re.sub('/',' ',bd)
            data.append(bd.strip()) #去掉前后空格
            
            datalist.append(data)
            #print(link)
    print(datalist)        
    return datalist

#得到指定URL网页的内容
def askURL(url):
    #用户代离 -- 本质告诉浏览器我们可以接受什么水平的文件
    #head内字典的格式一定要严格 不能有空格
    head = {
       "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.70"
    }

    request = urllib.request.Request(url,headers=head)
    try:
        response_0 = urllib.request.urlopen(request)
        html = response_0.read().decode("utf-8")
    #    print(html)        
    except Exception as e:       
        if hasattr(e,"code"):
            print(r.code)
        if hasattr(e,"reason"):
            print(e.reason)
    
    return html

#3.保存数据
def saveData(savepath):
    pass
    
    
if __name__ == '__main__':
    main()

[['https://movie.douban.com/subject/1292052/', 'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p480747492.jpg', '肖申克的救赎', '\xa0\xa0The Shawshank Redemption', '9.7', '2399806', '希望让人自由', '导演: 弗兰克·德拉邦特 Frank Darabont\xa0\xa0\xa0主演: 蒂姆·罗宾斯 Tim Robbins  ... 1994\xa0 \xa0美国\xa0 \xa0犯罪 剧情'], ['https://movie.douban.com/subject/1291546/', 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p2561716440.jpg', '霸王别姬', ' ', '9.6', '1786477', '风华绝代', '导演: 陈凯歌 Kaige Chen\xa0\xa0\xa0主演: 张国荣 Leslie Cheung   张丰毅 Fengyi Zha... 1993\xa0 \xa0中国大陆 中国香港\xa0 \xa0剧情 爱情 同性'], ['https://movie.douban.com/subject/1292720/', 'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p2372307693.jpg', '阿甘正传', '\xa0\xa0Forrest Gump', '9.5', '1806877', '一部美国近现代史', '导演: 罗伯特·泽米吉斯 Robert Zemeckis\xa0\xa0\xa0主演: 汤姆·汉克斯 Tom Hanks   ... 1994\xa0 \xa0美国\xa0 \xa0剧情 爱情'], ['https://movie.douban.com/subject/1295644/', 'https://img2.doubanio.com/view/photo/s_ratio_poster/public/p511118051.jpg', '这个杀手不太冷', '

# BeautifulSoup补充 

## 对象认识

+ BeautifulSoup4将复杂的HTML文档转换为一个复杂的树形结构，每个节点都是python对象，所有对象可以归纳为一下4种:
    + Tag:标签及其内容；拿到它所找到的第一个内容
    + NavigableString:标签里的内容 字符串
    + BeautifulSoup:整个文档
    + COmment:特殊的NavigableString,输出的内容不包含注释符号

In [2]:
from bs4 import BeautifulSoup
file = open("baidu.html","rb")
html = file.read()
#BeautifulSoup(对象，解析器)
bs = BeautifulSoup(html,"html.parser")
type(bs.head) #bs4.element.Tag

bs4.element.Tag

In [6]:
bs.title.string,type(bs.title.string) #bs4.element.NavigableString

('\r\n   百度一下，你就知道 ', bs4.element.NavigableString)

In [7]:
bs.a.attrs

{'class': ['mnav'], 'href': 'http://news.baidu.com', 'name': 'tj_trnews'}

In [8]:
type(bs) #bs4.BeautifulSoup

bs4.BeautifulSoup

In [12]:
type(bs.a.string)

bs4.element.NavigableString

## 文档的遍历 


In [14]:
#contents 将tag子节点以列表的形式输出
print(bs.head.contents)
print(bs.head.contents[1])
#children 获取tag的所有子节点 返回一个生成器

['\n', <meta content="text/html;charset=utf-8" http-equiv="content-type"/>, '\n', <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>, '\n', <meta content="always" name="referrer"/>, '\n', <link href="https://ss1.bdstatic.com/5eN1bjq8AAUYm2zgoY3K/r/www/cache/bdorz/baidu.min.css" rel="stylesheet" type="text/css"/>, '\n', <title>
   百度一下，你就知道 </title>, '\n']
<meta content="text/html;charset=utf-8" http-equiv="content-type"/>


## 文档搜索  find_all()

### 字符串过滤：会查找与字符串完全匹配的内容 

In [16]:
t_list = bs.find_all("a")
t_list

[<a class="mnav" href="http://news.baidu.com" name="tj_trnews">
        新闻 </a>,
 <a class="mnav" href="https://www.hao123.com" name="tj_trhao123">
        hao123 </a>,
 <a class="mnav" href="http://map.baidu.com" name="tj_trmap">
        地图 </a>,
 <a class="mnav" href="http://v.baidu.com" name="tj_trvideo">
        视频 </a>,
 <a class="mnav" href="http://tieba.baidu.com" name="tj_trtieba">
        贴吧 </a>,
 <a class="bri" href="//www.baidu.com/more/" name="tj_briicon" style="display: block;">
        更多产品 </a>]

In [17]:
#正则表达式搜索：search()
#标签含有a全都显示
import re
t_list = bs.find_all(re.compile("a"))
t_list

[<head>
 <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
 <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
 <meta content="always" name="referrer"/>
 <link href="https://ss1.bdstatic.com/5eN1bjq8AAUYm2zgoY3K/r/www/cache/bdorz/baidu.min.css" rel="stylesheet" type="text/css"/>
 <title>
    百度一下，你就知道 </title>
 </head>,
 <meta content="text/html;charset=utf-8" http-equiv="content-type"/>,
 <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>,
 <meta content="always" name="referrer"/>,
 <a class="mnav" href="http://news.baidu.com" name="tj_trnews">
        新闻 </a>,
 <a class="mnav" href="https://www.hao123.com" name="tj_trhao123">
        hao123 </a>,
 <a class="mnav" href="http://map.baidu.com" name="tj_trmap">
        地图 </a>,
 <a class="mnav" href="http://v.baidu.com" name="tj_trvideo">
        视频 </a>,
 <a class="mnav" href="http://tieba.baidu.com" name="tj_trtieba">
        贴吧 </a>,
 <a class="bri" href="//www.baidu.com/more/" name="tj_briicon" style="displa

### kwargs 参数 

In [19]:
t_list = bs.find_all(id="head")

for temp in t_list:
    print(temp)

<div id="head">
<div class="head_wrapper">
<div id="u1">
<a class="mnav" href="http://news.baidu.com" name="tj_trnews">
       新闻 </a>
<a class="mnav" href="https://www.hao123.com" name="tj_trhao123">
       hao123 </a>
<a class="mnav" href="http://map.baidu.com" name="tj_trmap">
       地图 </a>
<a class="mnav" href="http://v.baidu.com" name="tj_trvideo">
       视频 </a>
<a class="mnav" href="http://tieba.baidu.com" name="tj_trtieba">
       贴吧 </a>
<a class="bri" href="//www.baidu.com/more/" name="tj_briicon" style="display: block;">
       更多产品 </a>
</div>
</div>
</div>


### text参数

+ 关于find_all(text=xx) 列表为空情况的出现问题解决方式：
+ 问题原因：编码方式
+ 解决方法：调用re模块（引用正则表达式）

In [40]:
#t_list = bs.find_all(text=(u"hao123")) 不能适用
t_list = bs.find_all(text=re.compile("\d"))

for temp in t_list:
    print(temp)


       hao123 


### limit参数
限定输出个数

In [41]:
t_list = bs.find_all('a',limit=3)

for temp in t_list:
    print(temp)

<a class="mnav" href="http://news.baidu.com" name="tj_trnews">
       新闻 </a>
<a class="mnav" href="https://www.hao123.com" name="tj_trhao123">
       hao123 </a>
<a class="mnav" href="http://map.baidu.com" name="tj_trmap">
       地图 </a>


### css选择器 

In [47]:
#t_list = bs.select('title') #通过标签来查找
#t_list = bs.select(".mnav") #类名
# t_list = bs.select("a[class='bri']") #通过属性来查找
# t_list = bs.select("head > title") #通过子标签

# t_list

[<title>
    百度一下，你就知道 </title>]

In [48]:
t_list = bs.select(".mnav~.bri")  #兄弟节点属性
t_list[0].get_text()

'\r\n       更多产品 '

## re模块

### 基础知识介绍 

![jupyter](./正则表达式.png)

![jupyter](./re库.png)

### 实战
**建议在正则表达式之前加上r 防止其被转义**

#### search() 

In [57]:
#正则表达式：字符串模式(判断其是否符合标准)
import re
pat = re.compile("AA") #创建模式对象(验证) AA--正则表达式
m = pat.search("AcBC") #none
m = pat.search("WEAA") #<re.Match object; span=(2, 4), match='AA'>
m = pat.search("AABCAA") #<re.Match object; span=(0, 2), match='AA'> 只找到第一个

<re.Match object; span=(0, 2), match='AA'>

In [59]:
#无模式对象
m = re.search("AA","ASAAA") #前面是规则 后面是对象
m

<re.Match object; span=(2, 4), match='AA'>

#### findall()

In [62]:
re.findall("[A-Z]+","ASDAEFEFsda")

['ASDAEFEF']

####  sub()

In [63]:
re.sub("a","A","asdaasdfasd") #找到a用A替换

'AsdAAsdfAsd'